# Data Preparation

In [1]:
import skimage as ski
import numpy as np

In [2]:
import sys
sys.path.append('../../DataPreparation')

import sys
sys.path.append('../../FeatureExtraction/SIFT')

import sys
sys.path.append('../../Saving & Loading Models')

In [3]:
from DataPreparation import preprocess_and_save_images

# Preprocess and save images
preprocess_and_save_images(start_index = 0, batch_size = 10)

Processing Scheherazade New: 100%|██████████| 10/10 [00:06<00:00,  1.50it/s]

Image preprocessing and saving completed.


In [4]:
from DataPreparation import load_images

# Load preprocessed images
x_data, y_data = load_images(start_index = 0, batch_size = 10)

Loading fonts: 100%|██████████| 4/4 [00:00<00:00,  5.71it/s]


# Feature Extraction

In [5]:
from SIFT import compute_sift_descriptors_to_train_Kmeans

sift_descriptors = compute_sift_descriptors_to_train_Kmeans(x_data)

Computing SIFT descriptors: 100%|██████████| 40/40 [00:19<00:00,  2.06it/s]


In [6]:
from SIFT import compute_sift_descriptors_to_train_SVM

sift_descriptors_svm = compute_sift_descriptors_to_train_SVM(x_data)

Computing SIFT descriptors: 100%|██████████| 40/40 [00:13<00:00,  2.93it/s]


In [7]:
from SIFT import train_kmeans

kmeans = train_kmeans(sift_descriptors, "MiniBatchKMeans")

c:\Users\fatom\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


#### Save Kmeans Model
After training the Kmeans model, we save it for future use.

In [8]:
from SavingLoadingModels import save_Kmeans_model

# Save the KMeans model
save_Kmeans_model(kmeans)

In [9]:
from SIFT import compute_histograms

feature_vectors = compute_histograms(kmeans, sift_descriptors_svm)

Computing histograms: 100%|██████████| 40/40 [00:01<00:00, 33.48it/s]


# Model Training

#### Prepare Data for SVM
We then scale our feature vectors using the StandardScaler from sklearn. This is a good practice before using SVM.

In [10]:
from SVM import prepare_data_for_svm

scaled_feature_vectors = prepare_data_for_svm(feature_vectors)

#### Train SVM
We then train our SVM classifier using the scaled feature vectors. We're using a Radial Basis Function (RBF) kernel.

In [11]:
from SVM import train_svm

clf = train_svm(scaled_feature_vectors, y_data)

#### Save SVM Model
After training the SVM model, we save it for future use.

In [16]:
from SavingLoadingModels import save_SVM_model

# Save the SVM model
save_SVM_model(clf)

#### Test SVM
Finally, we test our trained SVM model on the test data and print the score.

In [17]:
from DataPreparation import load_images
from SVM import test_svm

# Load preprocessed images
x_test, y_test = load_images(start_index = 10, batch_size = 2)

# Compute the feature vectors for the test set
feature_vectors_test = compute_histograms(kmeans, compute_sift_descriptors_to_train_SVM(x_test))

# Test the SVM
score = test_svm(clf, feature_vectors_test, y_test)

print("Score: ", score)

Computing histograms: 100%|██████████| 8/8 [00:00<00:00, 67.67it/s]

Score:  0.25


# Load Model

#### Load KMeans Model
We can load the saved KMeans model when we need it.

In [18]:
from SavingLoadingModels import load_Kmeans_model

# Load the KMeans model
kmeans = load_Kmeans_model()

## Load SVM Model
We can load the saved SVM model when we need it.

In [19]:
from SavingLoadingModels import load_SVM_model

# Load the SVM model
clf = load_SVM_model()